In [ ]:
import sys
import numpy as np
sys.path.append('..')

from matplotlib import pyplot as plt

import MeshFEM
import mesh, tri_mesh_viewer

In [ ]:
#m = mesh.Mesh('../../misc/examples/meshes/ball.msh').boundaryMesh()
m = mesh.Mesh('../../misc/examples/meshes/sphere_hires.msh')

In [ ]:
v = tri_mesh_viewer.TriMeshViewer(m, wireframe=True, width=1024, height=768)
v.show()

In [ ]:
import elastic_sheet, energy, tensors

et = tensors.ElasticityTensor2D(200, 0.3)
psi = energy.StVenantKirchhoffCBased(et)

es = elastic_sheet.ElasticSheet(m, psi)

In [ ]:
def visualizeCurvature(sheet, view = None, which=None):
    pcurv = sheet.getPrincipalCurvatures()
    visField = which
    if visField is None: visField = 0
    if view is not None: view.update(preserveExisting=False, scalarField=pcurv[:, visField])
    if which is None:
        plt.hist(pcurv[:, 0], bins=100, alpha=0.75)
        plt.hist(pcurv[:, 1], bins=100, alpha=0.75)
    else:
        plt.hist(pcurv[:, which], bins=100, alpha=0.75)
    plt.show()

In [ ]:
visualizeCurvature(es, v)

In [ ]:
# Verify the effect of uniform scaling: curvatures should scale by the reciprocal of the scale factor
es.setDeformedPositions(es.getDeformedPositions() * 2.0)

In [ ]:
visualizeCurvature(es)

In [ ]:
# Test curvature of a flat plate wrapped into a cylinder.

In [ ]:
import triangulation
pts = [[0, 0], [0, 1], [1, 1], [1, 0]]
edges = [[0, 1], [1, 2], [2, 3], [3, 0]]
m_square = mesh.Mesh(*triangulation.triangulate(pts, edges, triArea=0.0001)[0:2], embeddingDimension=3)
plate = elastic_sheet.ElasticSheet(m_square, psi)

In [ ]:
plate.setIdentityDeformation()
X = plate.getDeformedPositions()
x = np.column_stack((np.sin(2 * np.pi * X[:, 0]) / (2 * np.pi), X[:, 1], np.cos(2 * np.pi * X[:, 0]) / (2 * np.pi)))
plate.setDeformedPositions(x)

Using the averaged midedge normals gives a pretty lousy discrete shape operator. However, the approximation is excellent if we infer the midedge
normals by minimizing the squared Frobenius norm of the second fundamental form (which promotes a smoother shape operator field over the surface, removing the mesh-dependent fluctuations.)

In [ ]:
plate.initializeMidedgeNormals(False) # re-initialize the midedge normals based on the deformed midsurface
visualizeCurvature(plate)

In [ ]:
plate.initializeMidedgeNormals(True) # re-initialze the midedge normals by minimizing the (integrated) second fundamental form's squared Frobenius norm
visualizeCurvature(plate)

In [ ]:
v_plate = tri_mesh_viewer.TriMeshViewer(plate, wireframe=True)
v_plate.show()

In [ ]:
plate_normalVis = tri_mesh_viewer.PointCloudViewer(plate.edgeMidpoints(), vectorField=plate.midedgeNormals(), superView=v_plate)
plate_normalVis.arrowSize = 20
visualizeCurvature(plate, v_plate)

In [ ]:
pcurv = plate.getPrincipalCurvatures()
np.median(pcurv, axis=0)

In [ ]:
import scipy.spatial
plate.applyRigidTransform(scipy.spatial.transform.Rotation.from_rotvec(np.random.normal(size=3)).as_matrix(), np.random.normal(size=3))
pcurvRot = plate.getPrincipalCurvatures()
pcurv - pcurvRot

In [ ]:
plate.prepareRigidMotionPins()

In [ ]:
plate.prepareRigidMotionPins()

In [ ]:
plate.getVars()[plate.prepareRigidMotionPins()]

In [ ]:
pcurvRot = plate.getPrincipalCurvatures()
np.linalg.norm((pcurv - pcurvRot))

Compare with curvatures computed when the cylindrical surface is used as the rest configuration.

In [ ]:
m_cylinder = m_square.copy()
m_cylinder.setVertices(x)
restPlate = elastic_sheet.ElasticSheet(m_cylinder, psi)
restPCurv = restPlate.getPrincipalCurvatures()

In [ ]:
np.median(restPCurv, axis=0)

In [ ]:
np.max(np.abs(pcurv - restPCurv))